In [3]:
import cv2  
from matplotlib import pyplot as plt

In [5]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import random

In [ ]:
from frame import TrpDataSet

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "CPU")
torch.cuda.empty_cache()

In [ ]:
wfd = WFDriver(r"C:\Users\lucyc\Desktop\faces\wider_face_split\wider_face_split\wider_face_train.mat",
               r"C:\Users\lucyc\Desktop\faces\WIDER_train\WIDER_train\images")

In [16]:

mat = scipy.io.loadmat(r"C:\Users\lucyc\Desktop\faces\wider_face_split\wider_face_split\wider_face_train.mat")


In [18]:
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'blur_label_list', 'event_list', 'expression_label_list', 'face_bbx_list', 'file_list', 'illumination_label_list', 'invalid_label_list', 'occlusion_label_list', 'pose_label_list'])

In [57]:
mat["face_bbx_list"][0].shape

(1,)

In [164]:
wfd = WFDriver(r"C:\Users\lucyc\Desktop\faces\wider_face_split\wider_face_split\wider_face_train.mat",
               r"C:\Users\lucyc\Desktop\faces\WIDER_train\WIDER_train\images")

In [165]:
wfd.get_file_path(61,2)

'C:\\Users\\lucyc\\Desktop\\faces\\WIDER_train\\WIDER_train\\images\\61--Street_Battle\\61_Street_Battle_streetfight_61_382.jpg'

array([[223, 289,  32,  44],
       [357, 258,  20,  27],
       [411, 313,  18,  26],
       [346, 224,  11,  15],
       [298, 224,  15,  20],
       [193, 244,  15,  14],
       [223, 232,  12,  14],
       [135, 239,  26,  35],
       [ 86, 222,   7,   8],
       [100, 231,   6,   8],
       [176, 232,   8,  10],
       [389, 238,  10,  14],
       [474, 212,  12,  13],
       [ 16, 225,   9,  11]])